In [1]:
import torch

from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

In [2]:
from Model.FastSCNN import *

In [3]:
with open("params.yml") as file:
    params = load(file, Loader=Loader)
    
dataset_path = params["dataset_path"]   # Path at which the dataset is located
crop_height  = params["crop_height"]    # Height of cropped/resized input image
crop_width   = params["crop_width"]     # Width of cropped/resized input image
num_classes  = params["num_classes"]    # Number of classes

In [4]:
model = FastSCNN(image_height   = crop_height,
                 image_width    = crop_width,
                 image_channels = 3,
                 num_classes    = num_classes)

model.load_state_dict(torch.load("./checkpoints/best_model.pt"))

<All keys matched successfully>

In [5]:
dummy_input = torch.randn(4, 3, crop_height, crop_width, device='cpu')

In [6]:
torch.onnx.export(model, dummy_input, "FastSCNN.onnx", verbose=True)

/home/antoniojkim/antonio/PyTorch/Fast-SCNN/Model/FastSCNN.py:97: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return F.interpolate(x, list(map(int, size)), mode='nearest') # mode='bilinear', align_corners=True)
/home/antoniojkim/antonio/PyTorch/Fast-SCNN/Model/FastSCNN.py:196: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = F.interpolate(x, list(map(int, size)), mode='nearest') # mode='bilinear', align_corners=True)


graph(%input.1 : Float(4, 3, 960, 1920),
      %learning_to_downsample.0.0.weight : Float(32, 3, 3, 3),
      %learning_to_downsample.0.1.weight : Float(32),
      %learning_to_downsample.0.1.bias : Float(32),
      %learning_to_downsample.0.1.running_mean : Float(32),
      %learning_to_downsample.0.1.running_var : Float(32),
      %learning_to_downsample.0.1.num_batches_tracked : Long(),
      %learning_to_downsample.1.0.weight : Float(32, 1, 3, 3),
      %learning_to_downsample.1.1.weight : Float(32),
      %learning_to_downsample.1.1.bias : Float(32),
      %learning_to_downsample.1.1.running_mean : Float(32),
      %learning_to_downsample.1.1.running_var : Float(32),
      %learning_to_downsample.1.1.num_batches_tracked : Long(),
      %learning_to_downsample.1.3.weight : Float(48, 32, 1, 1),
      %learning_to_downsample.1.4.weight : Float(48),
      %learning_to_downsample.1.4.bias : Float(48),
      %learning_to_downsample.1.4.running_mean : Float(48),
      %learning_to_downsa

In [7]:
import onnx
from onnx import version_converter, helper

In [8]:
# load model
original_model  = onnx.load("FastSCNN.onnx")

# converts oppset v9 to v8
converted_model = original_model # version_converter.convert_version(original_model, 8)

# change attribute of all Upsample nodes
for node in converted_model.graph.node:
    if node.op_type == 'Upsample':
        # get id-attribute_name map
        id = { attribute.name: id for id, attribute in enumerate(node.attribute)}
        print(node)
        print(id)
        # get & remove "scales" attribute
        att_scales = node.attribute.pop(id['scales']) 
        _, _, scale_height, scale_width = att_scales.floats # CARE IT DEPENDS ON ORDER. HERE [B, C, W, H] IS EXPECTED
        # append new attributes 'scale_width' & 'scale_height'
        node.attribute.extend([
            helper.make_attribute('width_scale', scale_width),
            helper.make_attribute('height_scale', scale_height)
        ])

input: "365"
input: "366"
output: "367"
op_type: "Upsample"
attribute {
  name: "mode"
  s: "nearest"
  type: STRING
}

{'mode': 0}


KeyError: 'scales'

In [ ]:

# save
onnx.save(converted_model, result_path)